In [ ]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.7 MB/s eta 0:00:00


In [40]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider, Text, IntSlider, Dropdown, FloatText, Button, VBox, HBox, Layout

# ---------------------------------
# Lorenz system and RK2 method
# ---------------------------------
def lorenz(u, t, sigma, rho, beta):
    x, y, z = u
    dx_dt = sigma*(y - x)
    dy_dt = rho*x - y - x*z
    dz_dt = x*y - beta*z
    return np.array([dx_dt, dy_dt, dz_dt])

def rk_2_step(u, t, h, f, sigma, rho, beta):
    k1 = f(u, t, sigma, rho, beta)
    u_tilde = u + h*k1
    k2 = f(u_tilde, t + h, sigma, rho, beta)
    u_next = u + (h/2.0)*(k1 + k2)
    return u_next

def simulate_lorenz(u0, sigma=10.0, rho=28.0, beta=8.0/3.0, h=0.01, T=50.0):
    t_array = np.arange(0, T+h, h)
    u = np.zeros((len(t_array),3))
    u[0] = u0
    for i in range(len(t_array)-1):
        u[i+1] = rk_2_step(u[i], t_array[i], h, lorenz, sigma, rho, beta)
    return t_array, u

# ---------------------------------
# Single trajectory interactive
# ---------------------------------
def plot_lorenz_single(u0_str="[1.0, 1.0, 1.0]", sigma=10.0, rho=28.0, beta=8.0/3.0, h=0.01, T=50.0):
    u0 = eval(u0_str)
    t_array, sol = simulate_lorenz(u0, sigma, rho, beta, h, T)
    fig = plt.figure(figsize=(6,5))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(sol[:,0], sol[:,1], sol[:,2], lw=1)
    ax.set_title(f'Lorenz System (RK2 Method)\nσ={sigma}, ρ={rho}, β={beta}, h={h}, T={T}, IC={u0}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.tight_layout()
    plt.show()

# ---------------------------------
# Multiple initial conditions interactive
# ---------------------------------
def plot_lorenz_multi_IC(sigma=10.0, rho=28.0, beta=8.0/3.0, h=0.01, T=50.0,
                         ic_list_str="[1.0,1.0,1.0]; [1.01,1.0,1.0]; [-5.0,5.0,20.0]"):
    ic_strs = ic_list_str.split(';')
    ic_list = []
    for s in ic_strs:
        s = s.strip()
        ic = eval(s)
        ic_list.append(ic)

    fig = plt.figure(figsize=(6,5))
    ax = fig.add_subplot(111, projection='3d')
    for ic in ic_list:
        t_array, sol = simulate_lorenz(ic, sigma, rho, beta, h, T)
        ax.plot(sol[:,0], sol[:,1], sol[:,2], lw=1, label=f'IC={ic}')
    ax.set_title(f'Lorenz System - Multiple ICs\n(σ={sigma}, ρ={rho}, β={beta}, T={T}, h={h})')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.tight_layout()
    plt.show()

# ---------------------------------
# Parameter sweep: vary rho and plot multiple trajectories
# ---------------------------------
def plot_lorenz_param_sweep(u0_str="[1.0, 1.0, 1.0]",
                            sigma_list_str="10,20,28,35",
                            beta_list_str="2.0,8.0/3.0,3.0,4.0",
                            rho_list_str="10,20,28,35",
                            h=0.01, T=50.0):
    u0 = eval(u0_str)
    sigma_values = [float(val.strip()) for val in sigma_list_str.split(',')]
    beta_values = [float(val.strip()) for val in beta_list_str.split(',')]
    rho_values = [float(val.strip()) for val in rho_list_str.split(',')]

    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')

    # Color cycle for distinguishing different parameter combinations
    color_cycle = plt.cm.get_cmap('tab10')

    param_combinations = [(sigma, beta, rho)
                          for sigma in sigma_values
                          for beta in beta_values
                          for rho in rho_values]

    for i, (sigma, beta, rho) in enumerate(param_combinations):
        t_array, sol = simulate_lorenz(u0, sigma, rho, beta, h, T)
        color = color_cycle(i % 10)
        ax.plot(sol[:,0], sol[:,1], sol[:,2],
                lw=1,
                label=f'σ={sigma}, β={beta}, ρ={rho}',
                color=color)

    ax.set_title(f'Lorenz System Parameter Sweep\nIC={u0}, T={T}, h={h}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Adjust legend to be outside the plot to prevent overcrowding
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    plt.tight_layout()
    plt.show()

# Interactive widgets
def interactive_single():
    return interact(
        plot_lorenz_single,
        u0_str=Text(value='[1.0,1.0,1.0]', description='IC'),
        sigma=FloatSlider(min=0, max=50.0, step=0.1, value=10.0, description='σ'),
        rho=FloatSlider(min=0, max=50.0, step=0.1, value=28.0, description='ρ'),
        beta=FloatSlider(min=0, max=10.0, step=0.1, value=8.0/3.0, description='β'),
        h=FloatSlider(min=0.001, max=0.5, step=0.001, value=0.001, description='h'),
        T=FloatSlider(min=10, max=200, step=1, value=50, description='T')
    )

def interactive_multi_ic():
    return interact(
        plot_lorenz_multi_IC,
        sigma=FloatSlider(min=0.1, max=50.0, step=0.1, value=10.0, description='σ'),
        rho=FloatSlider(min=0.1, max=50.0, step=0.1, value=28.0, description='ρ'),
        beta=FloatSlider(min=0.1, max=10.0, step=0.1, value=8.0/3.0, description='β'),
        h=FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='h'),
        T=FloatSlider(min=10, max=200, step=1, value=50, description='T'),
        ic_list_str=Text(value='[1.0,1.0,1.0]; [1.01,1.0,1.0]; [-5.0,5.0,20.0]', description='ICs')
    )

def interactive_param_sweep():
    return interact(
        plot_lorenz_param_sweep,
        u0_str=Text(value='[1.0,1.0,1.0]', description='IC'),
        sigma_list_str=Text(value='10,20,28,35', description='σ list'),
        beta_list_str=Text(value='2.0,8.0/3.0,3.0,4.0', description='β list'),
        rho_list_str=Text(value='10,20,28,35', description='ρ list'),
        h=FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='h'),
        T=FloatSlider(min=10, max=200, step=1, value=50, description='T')
    )

def display_guide():
    print("### Lorenz System Experiments (Heun's RK2 Method) ###\n")
    print("Use the following functions:")
    print("interactive_single()   -> Explore a single trajectory with adjustable parameters and IC")
    print("interactive_multi_ic() -> Explore multiple initial conditions on one plot")
    print("interactive_param_sweep() -> Perform a parameter sweep for rho values")
    print("\nObservations:")
    print("- For σ=10, ρ=28, β=8/3: Classic chaotic Lorenz attractor.")
    print("- Vary IC to see sensitivity to initial conditions.")
    print("- Change ρ to <1 to see convergence to origin, or other values to see non-chaotic behavior.")
    print("- Increase T to observe long-term non-periodic behavior around the attractor.")
    print("- Decrease h for more accuracy at higher computational cost.")

display_guide()

### Lorenz System Experiments (Heun's RK2 Method) ###

Use the following functions:
interactive_single()   -> Explore a single trajectory with adjustable parameters and IC
interactive_multi_ic() -> Explore multiple initial conditions on one plot
interactive_param_sweep() -> Perform a parameter sweep for rho values

Observations:
- For σ=10, ρ=28, β=8/3: Classic chaotic Lorenz attractor.
- Vary IC to see sensitivity to initial conditions.
- Change ρ to <1 to see convergence to origin, or other values to see non-chaotic behavior.
- Increase T to observe long-term non-periodic behavior around the attractor.
- Decrease h for more accuracy at higher computational cost.


In [ ]:
interactive_single()

interactive(children=(Text(value='[1.0,1.0,1.0]', description='IC'), FloatSlider(value=10.0, description='σ', …

<function __main__.plot_lorenz_single(u0_str='[1.0, 1.0, 1.0]', sigma=10.0, rho=28.0, beta=2.6666666666666665, h=0.01, T=50.0)>

In [ ]:
interactive_multi_ic()

interactive(children=(FloatSlider(value=10.0, description='σ', max=50.0, min=0.1), FloatSlider(value=28.0, des…

<function __main__.plot_lorenz_multi_IC(sigma=10.0, rho=28.0, beta=2.6666666666666665, h=0.01, T=50.0, ic_list_str='[1.0,1.0,1.0]; [1.01,1.0,1.0]; [-5.0,5.0,20.0]')>

In [43]:
interactive_param_sweep()

interactive(children=(Text(value='[1.0,1.0,1.0]', description='IC'), Text(value='10,20,28,35', description='σ …

<function __main__.plot_lorenz_param_sweep(u0_str='[1.0, 1.0, 1.0]', sigma_list_str='10,20,28,35', beta_list_str='2.0,8.0/3.0,3.0,4.0', rho_list_str='10,20,28,35', h=0.01, T=50.0)>